# Function Calling Examples with Foundry Local

This notebook demonstrates how to use function calling capabilities with Foundry Local SDK and OpenAI API.

## Package Installation

Install the OpenAI package for API communication:

In [1]:
! pip install openai

Install the Foundry Local SDK for local model management:

In [2]:
! pip install foundry-local-sdk

## Setup and Configuration

Import the FoundryLocalManager for managing local models:

In [3]:
from foundry_local import FoundryLocalManager

Define the model alias that will be used throughout this example:

In [4]:
alias = "phi-4-mini"

Create a FoundryLocalManager instance using the specified model alias:

In [5]:
manager = FoundryLocalManager(alias)

Import the OpenAI library for API interactions:

In [6]:
import openai

Create an OpenAI client using the local endpoint and API key from the manager:

In [7]:
client = openai.OpenAI(
    base_url=manager.endpoint,
    api_key=manager.api_key  # API key is not required for local usage
)

## Function Definitions

Define the available tools/functions for the AI model. This includes flight booking and hotel booking functions:

In [8]:
tool_list = '[{"name": "booking_flight_tickets", "description": "booking flights", "parameters": {"origin_airport_code": {"description": "The name of Departure airport code", "type": "string"}, "destination_airport_code": {"description": "The name of Destination airport code", "type": "string"}, "departure_date": {"description": "The date of outbound flight", "type": "string"}, "return_date": {"description": "The date of return flight", "type": "string"}}}, {"name": "booking_hotels", "description": "booking hotel", "parameters": {"destination": {"description": "The name of the city", "type": "string"}, "check_in_date": {"description": "The date of check in", "type": "string"}, "checkout_date": {"description": "The date of check out", "type": "string"}}}]'

## Parallel Function Support

Create a chat completion request that will trigger multiple function calls (flight booking and hotel booking) in parallel:

In [9]:
stream = client.chat.completions.create(
    model=manager.get_model_info(alias).id,
    messages=[{"role": "user", "content": "book flight ticket from Beijing to Paris(using airport code) in 2025-12-04 to 2025-12-10 , then book hotel from 2025-12-04 to 2025-12-10 in Paris"}],
    tools=[{"name": "booking_flight_tickets", "description": "booking flights", "parameters": {"origin_airport_code": {"description": "The name of Departure airport code", "type": "string"}, "destination_airport_code": {"description": "The name of Destination airport code", "type": "string"}, "departure_date": {"description": "The date of outbound flight", "type": "string"}, "return_date": {"description": "The date of return flight", "type": "string"}}}, {"name": "booking_hotels", "description": "booking hotel", "parameters": {"destination": {"description": "The name of the city", "type": "string"}, "check_in_date": {"description": "The date of check in", "type": "string"}, "checkout_date": {"description": "The date of check out", "type": "string"}}}],
    temperature=0.00001,
    max_tokens=4096,
    top_p = 1.0,
    stream=True
)

Process and display the streaming response from the model:

In [10]:
for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="", flush=True)

functools[{"name": "booking_flight_tickets", "arguments": {"origin_airport_code": "PEK", "destination_airport_code": "CDG", "departure_date": "2025-12-04", "return_date": "2025-12-10"}}, {"name": "booking_hotels", "arguments": {"destination": "Paris", "check_in_date": "2025-12-04", "checkout_date": "2025-12-10"}}]

## Single Function Support

Create a chat completion request for a single function call (weather inquiry). Note: This example shows a different format for defining tools:

In [11]:
stream = client.chat.completions.create(
    model=manager.get_model_info(alias).id,
    messages=[{"role": "user", "content": "What is the weather today in Paris?"}],
    tools=[
      {
        "function": {
          "name": "get_current_weather",
          "arguments": {
            "format": "celsius",
            "location": "Paris"
          }
        }
      }
    ],
    temperature=0.00001,
    max_tokens=4096,
    top_p = 1.0,
    stream=True
)

Process and display the streaming response for the single function call:

In [12]:
for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="", flush=True)

functools[{"name": "get_current_weather", "arguments": {"format": "celsius", "location": "Paris"}}]